In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs, urlencode
import json
import lxml
import streamlit as st

import matplotlib.pyplot as plt

import os
import sys
from pathlib import Path

import bcrypt

from google.cloud import bigquery
from google.oauth2 import service_account
import gspread
from gspread_dataframe import set_with_dataframe
from googleapiclient.discovery import build
import io
from googleapiclient.http import MediaIoBaseDownload

import warnings

In [2]:
KEY_PATH = ".config/"
servicekey_path = KEY_PATH + "serviceKey.json" ## 빅쿼리 외 다른 API 활용 위해
bigquerykey_path = KEY_PATH + "mido-project-426906-31b49963ac97.json"
sheetskey_path = KEY_PATH + "mido-project-426906-41a4b6d0e3db.json"
midopluskey_path = KEY_PATH + "midoplus.json"

warnings.filterwarnings("ignore")


In [3]:
def get_service_key(servicekey_path):

    with open(servicekey_path) as f:
        data = json.load(f)
    return data

In [4]:
# BigQuery 클라이언트 생성 함수
def create_bigquery_client(key_path):
    credentials = service_account.Credentials.from_service_account_file(key_path)
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)
    return client


In [5]:
create_bigquery_client(midopluskey_path)

In [6]:
def save_dataframe_to_bigquery(df, dataset_id, table_id, key_path):
    # BigQuery 클라이언트 객체 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 데이터프레임을 BigQuery 테이블에 적재
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = "WRITE_TRUNCATE"  # 기존 테이블 내용 삭제 후 삽입

    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()  # 작업 완료 대기

    print(f"Data inserted into table {table_id} successfully.")

In [7]:
def get_dataframe_from_bigquery(dataset_id, table_id, key_path):
    # BigQuery 클라이언트 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 테이블 데이터를 DataFrame으로 변환
    df = client.list_rows(table_ref).to_dataframe()

    return df

In [8]:
# 비밀번호 생성 및 해싱 함수
def generate_hashed_password(phone_number):
    
    # 전화번호의 뒷자리 4자리로 비밀번호 생성
    password = phone_number[-4:]
    
    # # 비밀번호 해싱
    # password = bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt())
    return password

### 빅쿼리

In [9]:
shopping_df = get_dataframe_from_bigquery('DATA_WAREHOUSE', 'g2b_data', midopluskey_path)
shopping_df_fin = shopping_df.drop('collection_Date',axis=1)

shopping_prod_df = get_dataframe_from_bigquery('DATA_MARTS', 'g2b_prod_data', midopluskey_path)
shopping_prod_df_fin = shopping_prod_df.fillna('')

news_df = get_dataframe_from_bigquery('DATA_MARTS', 'news_data', midopluskey_path)
news_df_fin = news_df.drop('collection_Date',axis=1)

### 구글 스프레드 시트

In [10]:
# Google Sheets 클라이언트 생성
sheets_scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/spreadsheets",
                "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

sheets_creds = service_account.Credentials.from_service_account_file(midopluskey_path, scopes=sheets_scope)
gc = gspread.authorize(sheets_creds)

### 지자체 교육청 예산 현황

In [12]:
# 스프레드시트 ID (URL에서 확인 가능)
business_sheet_id = '166xdkZYI-SDNwdEiI6-Kt-p1wsTwFTAYAVfuCiFly0E' ## 미도플러스 사업현황

In [13]:
# 기존 스프레드시트 및 워크시트 열기
WORKSHEET_NAME = '지자체'
spreadsheet_business = gc.open_by_key(business_sheet_id)
worksheet_business = spreadsheet_business.worksheet(WORKSHEET_NAME)

# 구글 시트에서 데이터 읽기
data = worksheet_business.get_all_records()
pd.DataFrame(data)

,지역명,자치단체명,세부사업명,삭제,예산현액,국비,시도비,시군구비,기타,지출액,편성액
0,강원특별자치도,원주시,간현생태공원 체육시설 조성(전환사업),FALSE,3000000000,0,1950000000,1050000000,0,0,3000000000
1,강원특별자치도,강릉시,"강릉 종합스포츠 타운 건립 (축구장, 야구장, 파크골프장, 테니스장)",FALSE,,,,,,,
2,강원특별자치도,강릉시,강릉스케이트장 인조잔디 설치,FALSE,,,,,,,
3,강원특별자치도,강릉시,강릉테니스장 조성사업,FALSE,140405840,0,0,140405840,0,138596600,0
4,강원특별자치도,춘천시,게이트볼장 시설개선,FALSE,85000000,0,85000000,0,0,84950700,0
...,...,...,...,...,...,...,...,...,...,...,...
708,충청북도,청주시,청주야구장 시설개선 사업,FALSE,1114817330,0,0,1114817330,0,1109201320,0
709,충청북도,옥천군,체육시설 부지조성사업,FALSE,1717642210,0,0,1717642210,0,1567759980,400000000
710,충청북도,단양군,체육시설물 유지관리 - 매포생활체육공원 축구장 인조잔디 교체공사 실시설계,FALSE,,,,,,,
711,충청북도,제천시,체육진흥시설지원(남부지역 생활체육공원조성),FALSE,949708830,0,0,949708830,0,26773800,600000000


In [14]:
# 기존 스프레드시트 및 워크시트 열기
WORKSHEET_NAME = '교육청'
spreadsheet_edu = gc.open_by_key(business_sheet_id)
worksheet_edu = spreadsheet_edu.worksheet(WORKSHEET_NAME)

# 구글 시트에서 데이터 읽기
data = worksheet_edu.get_all_records()
pd.DataFrame(data)

,시도,시군구,구분,과업명,삭제,금액,면적,예산집행
0,강원특별자치도,교육부,국립,강원대학교사범대학부설고등학교 인조잔디운동장조성 및 환경개선,FALSE,2442350000,,
1,강원특별자치도,강릉교육지원청,공립,경포중학교 야구장 시설개선,FALSE,10900000,9298,학교
2,강원특별자치도,강릉교육지원청,공립,관동중학교 인조잔디 운동장 조성,FALSE,806400000,4082,교육청
3,강원특별자치도,춘천교육지원청,공립,봉의초등학교 테니스장교체,FALSE,613290000,,
4,강원특별자치도,화천교육지원청,공립,상서중학교 운동장 보수,FALSE,89533000,4664,학교
...,...,...,...,...,...,...,...,...
156,충청남도,예산교육지원청,공립,예산여자중학교 유해운동장 교체,FALSE,259105000,2900,교육청
157,충청남도,충청남도교육청,공립,합덕제철고등학교 운동장 교체,FALSE,42486000,,
158,충청남도,논산계룡교육지원청,공립,강경여자중학교 인조잔디 교체,FALSE,150144000,5770,교육청
159,충청북도,청주교육지원청,공립,서경중학교 운동장 보수 및 바닥교체,FALSE,602871000,3500,교육청


In [15]:
# # 새로운 시트 생성
# budget_df = pd.DataFrame(data)
# new_sheet_name = '지자체백업'
# new_sheet = spreadsheet_business.add_worksheet(title=new_sheet_name, rows=len(budget_df), cols=len(budget_df.columns))

# set_with_dataframe(new_sheet, budget_df)

In [16]:
# # 데이터프레임을 기존 워크시트에 업로드
# budget_df = pd.DataFrame(data)
# set_with_dataframe(worksheet, budget_df)

### 종합쇼핑몰 납품 상세

In [11]:
# 스프레드시트 ID (URL에서 확인 가능)
shopping_sheet_id = '16vld1WTJwsrWPD_kFcUX1LJb-9WAtY2gnPFG5nGEqZ0' ## 미도플러스 종합쇼핑몰 현황
spreadsheet_shopping = gc.open_by_key(shopping_sheet_id)

In [12]:
# 기존 스프레드시트 및 워크시트 열기
WORKSHEET_NAME = '납품현황'
worksheet_shopping = spreadsheet_shopping.worksheet(WORKSHEET_NAME)

# 기존 데이터 지우기
worksheet_shopping.clear()

# 데이터 프레임을 스프레드 시트에 덮어쓰기
worksheet_shopping.update([shopping_df_fin.columns.values.tolist()] + shopping_df_fin.values.tolist())

{'spreadsheetId': '16vld1WTJwsrWPD_kFcUX1LJb-9WAtY2gnPFG5nGEqZ0',
 'updatedRange': "'납품현황'!A1:AL14346",
 'updatedRows': 14346,
 'updatedColumns': 38,
 'updatedCells': 545148}

### 종합쇼핑몰 품목 정보

In [17]:
# 스프레드시트 ID (URL에서 확인 가능)
shopping_sheet_id = '16vld1WTJwsrWPD_kFcUX1LJb-9WAtY2gnPFG5nGEqZ0' ## 미도플러스 종합쇼핑몰 현황
spreadsheet_shopping = gc.open_by_key(shopping_sheet_id)

In [18]:
# 기존 스프레드시트 및 워크시트 열기
WORKSHEET_NAME = '품목정보'
worksheet_shopping_prod = spreadsheet_shopping.worksheet(WORKSHEET_NAME)

# 기존 데이터 지우기
worksheet_shopping_prod.clear()

# 데이터 프레임을 스프레드 시트에 덮어쓰기
worksheet_shopping_prod.update([shopping_prod_df_fin.columns.values.tolist()] + shopping_prod_df_fin.values.tolist())

{'spreadsheetId': '16vld1WTJwsrWPD_kFcUX1LJb-9WAtY2gnPFG5nGEqZ0',
 'updatedRange': "'품목정보'!A1:AH624",
 'updatedRows': 624,
 'updatedColumns': 34,
 'updatedCells': 21216}

In [21]:
# # 데이터프레임을 기존 워크시트에 업로드
# budget_df = pd.DataFrame(data)
# set_with_dataframe(worksheet, budget_df)

### 뉴스스크랩

In [13]:
# 스프레드시트 ID (URL에서 확인 가능)
news_sheet_id = '1BnbDPv79Y44RpbMmZA2wFiFlM3mNN5xPtyOXFW0ABBk' ## midoplus news
spreadsheet_news = gc.open_by_key(news_sheet_id)

In [14]:
# 기존 스프레드시트 및 워크시트 열기
WORKSHEET_NAME = 'news'
worksheet_news = spreadsheet_news.worksheet(WORKSHEET_NAME)

# 기존 데이터 지우기
worksheet_news.clear()

# 데이터 프레임을 스프레드 시트에 덮어쓰기
worksheet_news.update([news_df_fin.columns.values.tolist()] + news_df_fin.values.tolist())

{'spreadsheetId': '1BnbDPv79Y44RpbMmZA2wFiFlM3mNN5xPtyOXFW0ABBk',
 'updatedRange': 'news!A1:D232',
 'updatedRows': 232,
 'updatedColumns': 4,
 'updatedCells': 928}

### 생산일지

#### 생산일지 업로드 로그

In [163]:
# 스프레드시트 ID (URL에서 확인 가능)
upload_sheet_id = '1sTpLxbmOdpGwSQDbvg02Eh1gxjVwSfI5PtMQ9sr-XmM' ## 계약관리, 생산일지 업로드
spreadsheet_upload = gc.open_by_key(upload_sheet_id)

In [164]:
# 기존 스프레드시트 및 워크시트 열기
WORKSHEET_NAME = 'upload_log'
worksheet_upload = spreadsheet_upload.worksheet(WORKSHEET_NAME)

# 구글 시트에서 데이터 읽기
data = worksheet_upload.get_all_records()
upload_log_df = pd.DataFrame(data)
upload_log_df

,계약코드,File Upload,업로드날짜
0,11111111,upload_log_Files_/11111111.File Upload.013056....,2024-08-19
1,22222222,upload_log_Files_/22222222.File Upload.013353....,2024-08-19
2,33333333,upload_log_Files_/33333333.File Upload.013854....,2024-08-19
3,122417736700,upload_log_Files_/122417736700.File Upload.015...,2024-08-19
4,240819,upload_log_Files_/240819.File Upload.074245.xlsx,2024-08-19
5,123123123123,upload_log_Files_/123123123123.File Upload.065...,2024-08-20
6,testtest,upload_log_Files_/testtest.File Upload.060212....,2024-08-21
7,testtest2,upload_log_Files_/testtest2.File Upload.062945...,2024-08-21


In [165]:
# 폴더 내 파일리스트 확인
# 특정 폴더 ID
folder_id = '1lUstrwPCzmeQDej4UyIwFB6tRIGpj04S' ## 'upload_log_Files_' 폴더id

# Google Drive API 클라이언트 생성
drive_service = build('drive', 'v3', credentials=sheets_creds)

# 폴더 내의 모든 파일 리스트 가져오기
results = drive_service.files().list(
    q=f"'{folder_id}' in parents",
    spaces='drive',
    fields="nextPageToken, files(id, name, mimeType, createdTime, modifiedTime)",
    # pageSize=100  # 한 번에 최대 100개의 파일을 가져옵니다.
).execute()

items = results.get('files', [])

# if not items:
#     print('No files found.')
# else:
#     print(f"Files in folder '{folder_id}':")
#     for item in items:
#         print(f"File Name: {item['name']}, File ID: {item['id']}, Created Time: {item['createdTime']}, Modified Time: {item['modifiedTime']}")

upload_file_df = pd.DataFrame(items)
upload_file_df

,mimeType,id,name,createdTime,modifiedTime
0,application/vnd.openxmlformats-officedocument....,1D0d6bJgqTXoYbR3VO1Kbxigv3A82UHOT,testtest2.File Upload.062945.xlsx,2024-08-21T06:29:46.378Z,2024-08-21T06:29:47.783Z
1,application/vnd.openxmlformats-officedocument....,1qGNMHJWW4K76Yee41iAwNquy-AoZDnx6,testtest.File Upload.060212.xlsx,2024-08-21T06:02:13.321Z,2024-08-21T06:02:14.784Z
2,application/vnd.openxmlformats-officedocument....,1rrYGA0S2eQqHKCOlOPbryIyHJLikFJao,123123123123.File Upload.065123.xlsx,2024-08-20T06:51:23.588Z,2024-08-20T06:51:25.120Z
3,application/vnd.openxmlformats-officedocument....,1TmNX3pP15YRbgvr2d4wvIldDuFnWpMti,240819.File Upload.074245.xlsx,2024-08-19T07:42:46.131Z,2024-08-19T07:42:47.283Z
4,application/vnd.openxmlformats-officedocument....,1I2IjS0Yy9di2Xkl5lib2jyxbhZQw5aXw,122417736700.File Upload.015042.xlsx,2024-08-19T01:50:43.447Z,2024-08-19T03:29:45.826Z
5,application/vnd.openxmlformats-officedocument....,1ad0JTmiaByNwffEWdXXtlTITEJR_GtR5,33333333.File Upload.013854.xlsx,2024-08-19T01:38:55.591Z,2024-08-19T01:38:56.971Z
6,application/vnd.openxmlformats-officedocument....,1tbMlhfbdTT5bSsiNYFl5VLgpzcje65yf,22222222.File Upload.013353.xlsx,2024-08-19T01:33:53.736Z,2024-08-19T01:33:55.113Z
7,application/vnd.openxmlformats-officedocument....,1MID0qw9YoBFUnlbWq853LbaDBu0R-6tT,11111111.File Upload.013056.xlsx,2024-08-19T01:30:58.123Z,2024-08-19T01:30:59.504Z


#### 업로드 생산일지 업데이트

In [166]:
# 최근 업로드된 파일 불러오기
recent_upload_nm = upload_log_df.iloc[-1]['File Upload'].split('/')[1]
file_id = upload_file_df[upload_file_df['name']==recent_upload_nm]['id'].iloc[0] ## 가장 최근 업로드한 파일 id

# 파일 다운로드
request = drive_service.files().get_media(fileId=file_id)
fh = io.BytesIO()
downloader = MediaIoBaseDownload(fh, request)
done = False

while not done:
    status, done = downloader.next_chunk()
    print(f"Download {int(status.progress() * 100)}% complete.")

# 메모리에서 파일 읽기
fh.seek(0)
upload_df = pd.read_excel(fh,header=5) ## 업로드된 생산일지 원본

Download 100% complete.


In [167]:
# 업로드된 생산일지 전처리
# 불필요데이터 제거
upload_df_fin = upload_df.iloc[:upload_df['R/NO'].last_valid_index()+1]

# 모든 컬럼 NaN 제거
upload_df_fin = upload_df_fin.dropna(how='all')

# 컬럼형식 맞추기
upload_df_fin = upload_df_fin.rename(columns={'날짜':'생산일자', '일자':'생산일자'})
upload_df_fin = upload_df_fin.rename(columns={'제직량 M':'제직량', '입고량 M':'입고량'})

# 일자, 구장명 채우기
upload_df_fin['생산일자'] = upload_df_fin['생산일자'].ffill()
upload_df_fin['구장명'] = upload_df_fin['구장명'].ffill()

# '오더량'부터 'YARN'까지의 컬럼을 '구장명' 기준 첫 행으로 채우기
if "계약코드" in upload_df_fin.columns:
    columns_to_fill = ['계약코드','오더량', 'ITEM', 'SPI', 'PH', 'PW', 'T/F TYPE', '기포지', '제직폭', '본수', '입고폭', 'YARN', 'COLOR', '도전사','R/NO']
else:
    columns_to_fill = ['오더량', 'ITEM', 'SPI', 'PH', 'PW', 'T/F TYPE', '기포지', '제직폭', '본수', '입고폭', 'YARN', 'COLOR', '도전사','R/NO']
for col in columns_to_fill:
    upload_df_fin[col] = upload_df_fin.groupby(['생산일자','구장명'])[col].transform(lambda x: x.fillna(x.iloc[0]))

# 제직량, 입고량 없는거 제거
upload_df_fin = upload_df_fin[~upload_df_fin['제직량'].isnull()].drop_duplicates().reset_index(drop=True)

# 스프레드시트 ID (URL에서 확인 가능)
production_sheet_id = '1dl6GyEkzarfgcKSmT9hwYzR63MhcGDox0z-gCa0_7OI' ## midoplus production_report --> 생산일지통합
spreadsheet_production = gc.open_by_key(production_sheet_id)

# 기존 스프레드시트 및 워크시트 열기
WORKSHEET_NAME = 'production_report'
worksheet_production = spreadsheet_production.worksheet(WORKSHEET_NAME)

# 구글 시트에서 데이터 읽기
data = worksheet_production.get_all_records()
production_df = pd.DataFrame(data)

# 기존 생산일지와 병합
production_df_fin = pd.concat([production_df,upload_df_fin],axis=0).reset_index(drop=True)
production_df_fin = production_df_fin.fillna('')

def convert_date_format(date_str):
    if isinstance(date_str, str):
        if len(date_str.split('.')) == 3:
            # 기존 형식이 '연도. 월. 일' 인 경우
            return pd.to_datetime(date_str, format='%Y.%m.%d').strftime('%Y-%m-%d')
        elif len(date_str.split('.')) == 2:
            # 새 형식이 '월.일' 인 경우, 연도를 '2024'로 가정
            return pd.to_datetime('2024.' + date_str, format='%Y.%m.%d').strftime('%Y-%m-%d')
    return date_str  # 변환할 수 없는 경우 또는 비문자형 데이터인 경우 원본 반환

# '생산일자' 컬럼에 변환 함수 적용
production_df_fin = production_df_fin.astype({'생산일자':str,'납품요구접수일자':str,'납품기한일자':str})
production_df_fin['생산일자'] = production_df_fin['생산일자'].str.split('~').str[0] 
production_df_fin['생산일자'] = production_df_fin['생산일자'].str.replace(' ','').str.replace('0:00','').str.replace('2024-05-078','2024-05-08') 
production_df_fin['생산일자'] = production_df_fin['생산일자'].apply(convert_date_format)
production_df_fin['납품요구접수일자'] = production_df_fin['납품요구접수일자'].str.replace(' ','').str.replace('0:00','')
production_df_fin['납품요구접수일자'] = production_df_fin['납품요구접수일자'].apply(convert_date_format)
production_df_fin['납품기한일자'] = production_df_fin['납품기한일자'].str.replace(' ','').str.replace('0:00','')
production_df_fin['납품기한일자'] = production_df_fin['납품기한일자'].apply(convert_date_format)

# 구글시트 업데이트 위해 NaN값 대체
production_df_fin = production_df_fin.fillna('')

# 계약코드 대체
production_df_fin['계약코드'] = np.where(production_df_fin['계약코드']=='',production_df_fin['납품요구번호'],production_df_fin['계약코드']) ## 계약코드 없는것 --> 납품요구번호로 대체
production_df_fin['계약코드'] = production_df_fin['계약코드'].astype(str)
production_df_fin['계약코드'] = production_df_fin['계약코드'].str.replace('-','')

# 제품 오입력 수정
production_df_fin['ITEM'] = production_df_fin['ITEM'].str.replace(',','')

# 납품요구건명 수정
production_df_fin['납품요구건명'] = production_df_fin['납품요구건명'].apply(lambda x: ' '.join(x.split('\n')[1].strip().split(' ')[1:]) if '\n' in x else x) ## 납품요구건명 오정보 수정

# 기존 데이터 지우기
worksheet_production.clear()

# 데이터 프레임을 스프레드 시트에 덮어쓰기
worksheet_production.update([production_df_fin.columns.values.tolist()] + production_df_fin.values.tolist())

{'spreadsheetId': '1dl6GyEkzarfgcKSmT9hwYzR63MhcGDox0z-gCa0_7OI',
 'updatedRange': 'production_report!A1:X9051',
 'updatedRows': 9051,
 'updatedColumns': 24,
 'updatedCells': 217224}